In [1]:
import torch

!pip list


Package                      Version
---------------------------- --------------------
absl-py                      1.4.0
anyio                        3.7.1
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
array-record                 0.4.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
cachetools                   5.3.1
certifi                      2022.12.7
cffi                         1.15.1
chardet                      4.0.0
charset-normalizer           3.2.0
click                        8.1.4
cmake                        3.26.4
comm                         0.1.3
contourpy                    1.1.0
cycler                       0.10.0
dbus-python                  1.2.16
debugpy                      1.6.7
decorator                    5.1.1
defusedxml                

In [4]:

torch.cuda.get_device_name(0)	#gpu 확인


'NVIDIA GeForce RTX 3060 Ti'

In [3]:
torch.cuda.is_available()	

True

In [1]:
if torch.cuda.is_available():
    print("GPU를 사용할 수 있습니다.")
else:
    print("GPU를 사용할 수 없습니다.")

NameError: name 'torch' is not defined

In [1]:
from ultralytics import YOLO

model = YOLO('/tf/notebook/runs/detect/train14/weights/last.pt')

image = '/tf/notebook/messageImage_1689833556852.jpg'

predictions = model.predict(image)

print(predictions)

get
# for prediction in predictions:
#     box_coordinates = prediction[:, :4]
#     class_probabilities = prediction[:, 4:]
#     #class_labels = prediction['클래스_레이블']
    
# print(box_coordinates, class_probabilities)


image 1/1 /tf/notebook/messageImage_1689833556852.jpg: 128x320 1 image_photo, 56.2ms
Speed: 1.6ms preprocess, 56.2ms inference, 8.0ms postprocess per image at shape (1, 3, 128, 320)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'typo_text', 1: 'typo_price_num', 2: 'typo_price_dollar', 3: 'typo_price_W', 4: 'typo_price_won_en', 5: 'typo_price_won_kr', 6: 'image_photo', 7: 'image_colorBG', 8: 'image_removeBG', 9: 'icon_arrow_left', 10: 'icon_arrow_top', 11: 'icon_arrow_bottom', 12: 'icon_arrow_right', 13: 'icon_video_play', 14: 'icon_SNS_insta', 15: 'icon_SNS_youtube', 16: 'btn_radius', 17: 'btn_ellipse', 18: 'btn_square'}
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        

In [15]:
box_coordinates = predictions.boxes.tensor[:, :4]
print(box_coordinates)

AttributeError: 'list' object has no attribute 'boxes'

In [17]:
# 예측 결과 리스트에서 바운딩 박스 좌표 추출
box_coordinates_list = [result.boxes.xyxy for result in predictions]

# 예측 결과 리스트의 각 원소에 대한 바운딩 박스 좌표 출력
for idx, box_coordinates in enumerate(box_coordinates_list):
    print(f"Prediction {idx+1} - Box Coordinates: {box_coordinates}")

Prediction 1 - Box Coordinates: tensor([[385.1505,   2.0778, 910.2239, 729.0000]], device='cuda:0')


In [18]:
def is_text_box_inside_image_box(image_box, text_box):
    image_x_min, image_y_min, image_x_max, image_y_max = image_box
    text_x_min, text_y_min, text_x_max, text_y_max = text_box
    
    if (text_x_min >= image_x_min and text_y_min >= image_y_min and
        text_x_max <= image_x_max and text_y_max <= image_y_max):
        return True
    else:
        return False

def is_text_on_right(image_box, text_box):
    image_x_max, image_y_min, _, image_y_max = image_box
    text_x_min, text_y_min, _, text_y_max = text_box

    if text_x_min >= image_x_max:
        return True
    else:
        return False

def is_text_on_left(image_box, text_box):
    image_x_min, image_y_min, _, image_y_max = image_box
    text_x_max, text_y_min, _, text_y_max = text_box

    if text_x_max <= image_x_min:
        return True
    else:
        return False    
    
def is_text_on_top(image_box, text_box):
    image_x_min, image_y_min, image_x_max, _ = image_box
    _, text_y_max, _, _ = text_box

    if text_y_max <= image_y_min:
        return True
    else:
        return False

def is_text_on_bottom(image_box, text_box):
    image_x_min, _, image_x_max, image_y_max = image_box
    _, _, _, text_y_min = text_box

    if text_y_min >= image_y_max:
        return True
    else:
        return False
    
def check_text_position(image_box, text_box):
    if is_text_box_inside_image_box(image_box, text_box):
        return "Inside"
    elif is_text_on_right(image_box, text_box):
        return "Right"
    elif is_text_on_top(image_box, text_box):
        return "Top"
    elif is_text_on_bottom(image_box, text_box):
        return "Bottom"
    else:
        return "Left"

In [20]:
# 예시 이미지 및 박스 좌표 설정
image_box = [0, 0, 800, 600]  # [x_min, y_min, x_max, y_max]
text_box = [800, 0, 1200, 500]  # [x_min, y_min, x_max, y_max]

# 텍스트 위치 체크
result = check_text_position(image_box, text_box)
print("Text Position:", result)

Text Position: Right


In [ ]:
def get_text_and_image_boxes(results):
    data = getModelResult(results)
    boxList = data['data']

    text_boxes = []
    image_boxes = []

    for box in boxList:
        class_name = box['class_name']
        if class_name == 'typo_text':
            text_boxes.append(box)
        elif class_name in ['image_photo', 'image_colorBG', 'image_removeBG']:
            image_boxes.append(box)

    return text_boxes, image_boxes

In [2]:
import os

def getModelResult( results ):
    ROOT_PATH = os.getcwd() + "\\";
    data = {}
    dataList = []
    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs
        names = result.names
        origImg = result.orig_shape
        imgWidth = origImg[1]
        imgHeight = origImg[0]

        data['img_width'] = imgWidth
        data['img_height'] = imgHeight
        print( origImg )
        # masks = result.masks  # Masks object for segmenation masks outputs
    #     probs = result.probs  # Class probabilities

    #     # print(boxes.xywhn)
    #     # print(boxes.cls)
    #     # print(masks)
    #     # print(probs)
        boxList = []
        for box in boxes:

            boxData = {}
            # box.xyxy[0]
            
            boxResult = box.xywhn.tolist()[0]
            classNum = int(box.cls.tolist()[0])
            className = names[classNum]
            conf = box.conf.tolist()[0]
            centerX = boxResult[0]
            centerY = boxResult[1]
            width = boxResult[2]
            height = boxResult[3]

            left = centerX - ( width / 2 )
            top = centerY - ( height / 2 )

            minX = left
            minY = top
            maxX = left + width
            maxY = top + height

            boxData['class_name'] = className
            boxData['class_id'] = classNum
            boxData['confidence'] = conf
            boxData['center_x'] = centerX
            boxData['center_y'] = centerY
            boxData['left'] = left
            boxData['top'] = top
            boxData['width'] = width
            boxData['height'] = height
            boxData['minX'] = minX
            boxData['minY'] = minY
            boxData['maxX'] = maxX
            boxData['maxY'] = maxY


            boxData['origin_center_x'] = centerX * imgWidth
            boxData['origin_center_y'] = centerY * imgHeight
            boxData['origin_left'] = left * imgWidth
            boxData['origin_top'] = top * imgHeight
            boxData['origin_width'] = width * imgWidth
            boxData['origin_height'] = height * imgHeight
            boxData['origin_minX'] = minX * imgWidth
            boxData['origin_minY'] = minY * imgHeight
            boxData['origin_maxX'] = maxX * imgWidth
            boxData['origin_maxY'] = maxY * imgHeight


            # if( boxData['confidence'] * 100 > 60 ):
            boxList.append( boxData )
        
        dataList.append( boxList )
    data['data'] = dataList
    # jsonStr = json.dumps( dataList )
    # print( jsonStr )
    return data

target_classes = ['typo_text','image_photo', 'image_colorBG', 'image_removeBG'] # 바운딩 박스를 추출하고자 하는 클래스명들의 리스트

def extract_boxes_by_class(boxList, target_classes):
    target_boxes = []  # target들을 넣을 박스 리스트
    for boxListPerImage in boxList:  # 각 이미지 별로 box 정보를 가져옴
        for box in boxListPerImage:  # 이미지 내의 모든 박스들을 반복하여 클래스 찾기
            class_name = box['class_name']
            if class_name in target_classes:
                target_boxes.append(box)  # 찾아냈으면 리스트에 추가
    return target_boxes



def get_text_and_image_boxes(results):
    boxList = results['data'][0]  # 'data' 리스트 안에 각 이미지의 바운딩 박스 정보가 들어있으므로 첫 번째 항목을 선택합니다.
    text_boxes = extract_boxes_by_class(boxList, ['typo_text'])
    image_boxes = extract_boxes_by_class(boxList, ['image_photo', 'image_colorBG', 'image_removeBG'])
    return text_boxes, image_boxes


def is_box_inside_image(text_boxes, image_boxes):                               # image label안에 text label이 있는지 체크
    

    for text_box in text_boxes:                                                 
        text_x_min, text_y_min, text_x_max, text_y_max = text_box 
        for image_box in image_boxes:
            image_x_min, image_y_min, image_x_max, image_y_max = image_box                                                 
            if not (text_x_min >= image_x_min and text_y_min >= image_y_min and
                    text_x_max <= image_x_max and text_y_max <= image_y_max):
                return False
    
    return True

def text_on_right_image(image_boxes, text_boxes):                               # 텍스트가 image 오른쪽에 있는지 체크

    for text_box in text_boxes:                                                 
        text_x_min, _, _, _ = text_box
        for image_box in image_boxes:
            image_x_max, _, _, _ = image_box   
        if text_x_min >= image_x_max:
            return True

    return False

def text_on_left_image(image_boxes, text_boxes):                                # 텍스트가 image 왼쪽에 있는지 체크
    
    for text_box in text_boxes:
        _, text_x_max, _, _ = text_box
        for image_box in image_boxes:
            image_x_min, _, _, _ = image_box
            if text_x_max <= image_x_min:
                return True
    
    return False


def text_on_bottom_image(image_boxes, text_boxes):                              # 텍스트가 image 아래쪽에 있는지 체크
    
    for text_box in text_boxes:
        _, _, _, text_y_min = text_box
        for image_box in image_boxes:
            _, _, _, image_y_max = image_box
            if text_y_min >= image_y_max:
                return True
    
    return False
                                                                                # 어느방향에 있는지 한번에 체크
def check_text_position(image_boxes, text_boxes):                               
    if is_box_inside_image(image_boxes, text_boxes):
        return "Image Inside Text"
    elif text_on_right_image(image_boxes, text_boxes):
        return "Image Right Text"
    elif text_on_left_image(image_boxes, text_boxes):
        return "Image Left Text"
    elif text_on_bottom_image(image_boxes, text_boxes):
        return "Image Bottom Text"
    else:
        return "Image and Texts are not related"
    
def get_min_max_coordinates(data):
    min_max_coordinates = []
    for image_boxes in data['data']:
        for box in image_boxes:
            minX = box['minX']
            minY = box['minY']
            maxX = box['maxX']
            maxY = box['maxY']
            min_max_coordinates.append((minX, minY, maxX, maxY))
    return min_max_coordinates

In [48]:
from ultralytics import YOLO

model = YOLO('/tf/notebook/runs/detect/train14/weights/last.pt')

image = '/tf/notebook/messageImage_1689833556852.jpg'

results = model.predict(image)


image 1/1 /tf/notebook/messageImage_1689833556852.jpg: 128x320 1 image_photo, 33.2ms
Speed: 0.6ms preprocess, 33.2ms inference, 1.8ms postprocess per image at shape (1, 3, 128, 320)


In [43]:

from ultralytics import YOLO

model = YOLO('/tf/notebook/runs/detect/train14/weights/last.pt')

image = '/tf/notebook/messageImage_1689833556852.jpg'

results = model.predict(image)

# extract_boxes_by_class 함수 호출
extracted_boxes_list = []
for result in results:
    data = getModelResult([result])  # getModelResult 함수는 리스트를 입력받으므로 현재 result를 리스트로 변환하여 호출
    boxlist = data['data'][0]  # 'data' 리스트 안에 각 이미지의 바운딩 박스 정보가 들어있으므로 첫 번째 항목을 선택합니다.
    extracted_boxes = extract_boxes_by_class(boxlist, target_classes)
    extracted_boxes_list.append(extracted_boxes)

# get_text_and_image_boxes 함수 호출
text_boxes, image_boxes = get_text_and_image_boxes(extracted_boxes_list)
print("Text Boxes:", text_boxes)
print("Image Boxes:", image_boxes)


image 1/1 /tf/notebook/messageImage_1689833556852.jpg: 128x320 1 image_photo, 8.6ms
Speed: 0.6ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 320)


(729, 2354)


TypeError: string indices must be integers

In [30]:
extracted_boxes_list = []
for result in results:
    data = getModelResult([result])  # getModelResult 함수는 리스트를 입력받으므로 현재 result를 리스트로 변환하여 호출
    boxlist = data['data']
    extracted_boxes = extract_boxes_by_class(boxlist, target_classes)
    extracted_boxes_list.append(extracted_boxes)

print(extracted_boxes_list)  # 각 이미지별로 추출된 바운딩 박스 확인|

(729, 2354)
[[{'class_name': 'image_photo', 'class_id': 6, 'confidence': 0.9632229208946228, 'center_x': 0.27514326572418213, 'center_y': 0.5014250874519348, 'left': 0.16361535340547562, 'top': 0.002850174903869629, 'width': 0.22305582463741302, 'height': 0.9971498250961304, 'minX': 0.16361535340547562, 'minY': 0.002850174903869629, 'maxX': 0.38667117804288864, 'maxY': 1.0, 'origin_center_x': 647.6872475147247, 'origin_center_y': 365.5388887524605, 'origin_left': 385.1505419164896, 'origin_top': 2.0777775049209595, 'origin_width': 525.0734111964703, 'origin_height': 726.922222495079, 'origin_minX': 385.1505419164896, 'origin_minY': 2.0777775049209595, 'origin_maxX': 910.2239531129599, 'origin_maxY': 729.0}]]


In [56]:
def get_boxes_list(data):
    boxes_list = []
    for image_boxes in data:
        boxes_list.append(image_boxes)
    return boxes_list


In [57]:
# getModelResult 함수를 사용하여 결과 데이터 얻기
results = model.predict(image)
data = getModelResult(results)

# 바운딩 박스 정보를 이중 리스트로 추출하기
boxes_list = get_boxes_list(data)
print(boxes_list)


image 1/1 /tf/notebook/messageImage_1689833556852.jpg: 128x320 1 image_photo, 59.9ms
Speed: 1.7ms preprocess, 59.9ms inference, 4.1ms postprocess per image at shape (1, 3, 128, 320)


(729, 2354)
['img_width', 'img_height', 'data']


In [53]:
for i in boxes_list:
    for j in i:
        print(j)

[6, 0.16361535340547562, 0.002850174903869629, 0.38667117804288864, 1.0]


In [60]:
print(results.boxes())

AttributeError: 'list' object has no attribute 'boxes'

In [7]:
def getModelResult(results):
    dataList = []
    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs
        names = result.names
        origImg = result.orig_shape
        imgWidth = origImg[1]
        imgHeight = origImg[0]

        data = []
        for box in boxes:
            boxResult = box.xywhn.tolist()[0]
            classNum = int(box.cls.tolist()[0])
            className = names[classNum]
            conf = box.conf.tolist()[0]
            centerX = boxResult[0]
            centerY = boxResult[1]
            width = boxResult[2]
            height = boxResult[3]

            left = centerX - (width / 2)
            top = centerY - (height / 2)

            minX = left
            minY = top
            maxX = left + width
            maxY = top + height

            boxData = [classNum, minX, minY, maxX, maxY]
            data.append(boxData)

        dataList.append(data)

    return dataList

# YOLO 모델로 이미지 예측 수행
results = model.predict(image)

# YOLO 결과를 바탕으로 바운딩 박스 정보 추출
boxes_list = getModelResult(results)
print(boxes_list[:,0])


image 1/1 /tf/notebook/messageImage_1689833556852.jpg: 128x320 1 image_photo, 108.1ms
Speed: 1.3ms preprocess, 108.1ms inference, 3.6ms postprocess per image at shape (1, 3, 128, 320)


TypeError: list indices must be integers or slices, not tuple

In [62]:
for i in results:
    print(i)

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'typo_text', 1: 'typo_price_num', 2: 'typo_price_dollar', 3: 'typo_price_W', 4: 'typo_price_won_en', 5: 'typo_price_won_kr', 6: 'image_photo', 7: 'image_colorBG', 8: 'image_removeBG', 9: 'icon_arrow_left', 10: 'icon_arrow_top', 11: 'icon_arrow_bottom', 12: 'icon_arrow_right', 13: 'icon_video_play', 14: 'icon_SNS_insta', 15: 'icon_SNS_youtube', 16: 'btn_radius', 17: 'btn_ellipse', 18: 'btn_square'}
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [

In [73]:
from ultralytics import YOLO

model = YOLO('/tf/notebook/runs/detect/train14/weights/last.pt')

image = '/tf/notebook/202307201123090707.jpg'

results = model.predict(image)

# 예측한 바운딩 박스 정보를 출력합니다.
for i, result in enumerate(results):
    print(f"Image {i+1}/{len(results)}")
    boxes = result.boxes.xyxy
    print("Bounding Box Coordinates:")
    print(boxes)

    class_ids = result.boxes.cls
    print("Class IDs:")
    print(class_ids)

    scores = result.boxes.conf
    print("Confidence Scores:")
    print(scores)
    print("--------------------")



image 1/1 /tf/notebook/202307201123090707.jpg: 160x320 5 image_photos, 45.6ms
Speed: 0.7ms preprocess, 45.6ms inference, 1.9ms postprocess per image at shape (1, 3, 160, 320)


Image 1/1
Bounding Box Coordinates:
tensor([[1087.4695,    1.9572, 1604.9768,  323.3430],
        [ 554.8138,    2.1431, 1081.7874,  401.3712],
        [1087.3472,  329.1396, 1603.8237,  726.0000],
        [  32.6036,    3.8265,  550.2618,  726.0000],
        [ 554.5333,  407.4024, 1082.8427,  726.0000]], device='cuda:0')
Class IDs:
tensor([6., 6., 6., 6., 6.], device='cuda:0')
Confidence Scores:
tensor([0.9796, 0.9788, 0.9771, 0.9732, 0.9541], device='cuda:0')
--------------------
